In [37]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime, date
import os
import errno
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf
from pathlib import Path

simpler_sf.simple_salesforce()

In [38]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [39]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
# regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [48]:

cargasBci = ("F:/Empresas SF/reportes/BCI.xlsx")
archivoBCI = pd.read_excel(cargasBci)

In [41]:
query2 = "select Id,Paciente__r.IdentificationId__pc from EmpresaContactoConvenios__c where Campa_a__c <> 'BCI Seguros Colectivos' and Campa_a__c  <> 'BCI Contingencia' and Campa_a__c <> 'BCI Seguros Colaboradores' and Campa_a__c <> 'BCI Seguros Individuales'"
df2 = sf.smart_query(query2)

100%|██████████| 1/1 [01:54<00:00, 114.62s/it]


In [49]:
archivoBCI["rut"] = archivoBCI["rut1"].astype(str)
archivoBCI["rut"] = archivoBCI["rut"].str.replace("-", "")
archivoBCI["rut"] = archivoBCI["rut"].str.replace("–", "")
archivoBCI["rut"] = archivoBCI["rut"].astype(str)
archivoBCI["rut"] = archivoBCI["rut"].str.replace(".", "")
archivoBCI["rut"] = archivoBCI["rut"].str.replace(" ", "")
archivoBCI["rut"] = archivoBCI["rut"].str.upper()
archivoBCI["rut"] = archivoBCI["rut"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11748\3830561832.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["rut"] = archivoBCI["rut"].str.replace(".", "")


In [50]:
df2 = df2.rename(
    columns={
        "Paciente__r.IdentificationId__pc": "rut",
        "Id":"Id",
        
    }
)

In [51]:
df2["rut"] = df2["rut"].astype(str)
df2["rut"] = df2["rut"].str.replace("-", "")
df2["rut"] = df2["rut"].str.replace("–", "")
df2["rut"] = df2["rut"].astype(str)
df2["rut"] = df2["rut"].str.replace(".", "")
df2["rut"] = df2["rut"].str.replace(" ", "")
df2["rut"] = df2["rut"].str.upper()
df2["rut"] = df2["rut"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11748\2389365936.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2["rut"] = df2["rut"].str.replace(".", "")


In [52]:
demases_empresas = pd.merge(
    left=archivoBCI,
    right=df2,
    how="left",
    left_on="rut",
    right_on="rut",
)

In [53]:
demases_empresas.head()

,PÓLIZA,EMPLEADOR,NOMBRE_GRUPO,rut2,gbng,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,RELACIÓN,rut,Id
0,139174,BCI SEGUROS GENERALES SA,BCI VIDA UF 1000,6041679.0,6450379,0,6450379-0,LILA DIAZ VASQUEZ,Esposa,64503790,NaN
1,139174,BCI SEGUROS GENERALES SA,BCI VIDA UF 1000,6041679.0,6041679,6,6041679-6,MARCOS EUGENIO CASTILLO MOLINA,Titular Masculino,60416796,NaN
2,139174,BCI SEGUROS GENERALES SA,BCI GENERALES 3000,7055269.0,7055269,8,7055269-8,ROBERTO JUAN HARAMBOURE GALAZ,Titular Masculino,70552698,NaN
3,139174,BCI SEGUROS GENERALES SA,BCI GENERALES 2000,8702571.0,9127571,6,9127571-6,ROSA PALACIOS MORENO,Esposa,91275716,NaN
4,139174,BCI SEGUROS GENERALES SA,BCI GENERALES 2000,8702571.0,8702571,3,8702571-3,GUILLERMO COFRE MAC NAMARA,Titular Masculino,87025713,NaN


In [54]:
demases_empresas.to_excel("F:/Empresas SF/reportes/demases.xlsx", index=False)